In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from simtk import unit
import parmed as pmd

def generate_gromacs_files(input_file, output_prefix="molecule",
                           ff_name="openff-2.0.0.offxml"):
    """
    Generate GROMACS topology (.top) and coordinates (.gro) using OpenFF + ParmEd.

    Parameters
    ----------
    input_file : str
        Input structure file (.mol2, .sdf, or .pdb)
    output_prefix : str
        Prefix for output files
    ff_name : str
        OpenFF force field name (e.g. 'openff-2.0.0.offxml')
    """

    # Load molecule (OpenFF can read from PDB, MOL2, SDF, or SMILES)
    molecule = Molecule.from_file(input_file)
    print(f"Loaded molecule: {molecule.to_smiles()} with {molecule.n_atoms} atoms")

    # Create topology and apply FF
    topology = Topology.from_molecules(molecule)
    forcefield = ForceField(ff_name)
    omm_system = forcefield.create_openmm_system(topology)

    # Convert to ParmEd structure
    structure = pmd.openmm.load_topology(
        topology.to_openmm(),
        omm_system,
        xyz=molecule.conformers[0].value_in_unit(unit.nanometer) if molecule.conformers else None
    )

    # Export to GROMACS format
    top_file = f"{output_prefix}.top"
    gro_file = f"{output_prefix}.gro"

    structure.save(top_file, overwrite=True)
    structure.save(gro_file, overwrite=True)

    print(f"\n✅ GROMACS-compatible files generated:")
    print(f" - Topology: {top_file}")
    print(f" - Coordinates: {gro_file}")
    print(f" - Force Field: {ff_name}")

    return top_file, gro_file


if __name__ == "__main__":
    # Example usage:
    generate_gromacs_files("molecule.pdb", output_prefix="molecule",
                           ff_name="openff-2.0.0.offxml")


ModuleNotFoundError: No module named 'openff'